In [1]:
!pip install tqdm

In [2]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.layers as layers 
import tensorflow.keras.models as models
import pandas as pd
import matplotlib.pyplot as plt
import tqdm

In [3]:
class Actor(models.Model):
    def __init__(self):
        super(Actor, self).__init__()
        
        self.cnn = models.Sequential([
            layers.Conv2D(16, 5, activation='relu'),
            layers.MaxPooling2D(),
            layers.Conv2D(32, 3, activation='relu'),
            layers.MaxPooling2D(),
            layers.BatchNormalization(),
            layers.Conv2D(32, 3, activation='relu'),
            layers.MaxPooling2D(),
            layers.Flatten()
        ])
        
        self.concat = layers.Concatenate(axis=1)
        
        self.dense = models.Sequential([
            layers.Dense(256, activation='relu'),
            layers.Dense(256, activation='relu'),
            layers.Dense(256, activation='relu'),
            
            layers.Dense(1, activation='tanh')
        ])
        
    def call(self, inputs):
        imgs, vels = inputs
        imgsx = self.cnn(imgs)
        x = self.concat([imgsx, vels])
        x = self.dense(x)
        return x


In [4]:
picPath = "../input/simple-car-simulator-dataset/CarSIM/PIC/"
csvPath = "../input/simple-car-simulator-dataset/CarSIM/Data.csv"

In [5]:
data = pd.read_csv(csvPath)
data.head()

,PicID,Vertical,Horizontal,Velocity
0,020721-010829-248-0,0,0,0.000000
1,020721-010829-781-1,0,0,0.548092
2,020721-010829-806-2,0,0,0.023570
3,020721-010829-934-3,0,0,0.022755
4,020721-010830-072-4,0,0,0.002016


In [6]:
pics = np.empty((len(data), 256, 256, 3), dtype=np.float32)
velocities = np.empty((len(data), 1), dtype=np.float32)
controls = np.empty((len(data), 2), dtype=np.float32)
for i in tqdm.tqdm(range(len(data))):
    name = picPath+data['PicID'][i] + ".png"
    pics[i] = plt.imread(name)[:,:,:3]
    velocities[i] = [data['Velocity'][i]]
    controls[i] = (data['Vertical'][i], data['Horizontal'][i])
    
    

100%|██████████| 2050/2050 [00:14<00:00, 138.99it/s]


In [7]:
model = Actor()

In [8]:
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit((pics, velocities), controls, )

65/65 [==============================] - 8s 62ms/step - loss: 0.5127


In [9]:
model.fit((pics, velocities), controls, epochs=15)

Epoch 1/15
65/65 [==============================] - 4s 60ms/step - loss: 0.3856
Epoch 2/15
65/65 [==============================] - 4s 62ms/step - loss: 0.3674
Epoch 3/15
65/65 [==============================] - 4s 61ms/step - loss: 0.3639
Epoch 4/15
65/65 [==============================] - 4s 62ms/step - loss: 0.3524
Epoch 5/15
65/65 [==============================] - 4s 61ms/step - loss: 0.3474
Epoch 6/15
65/65 [==============================] - 4s 62ms/step - loss: 0.3404
Epoch 7/15
65/65 [==============================] - 4s 61ms/step - loss: 0.3357
Epoch 8/15
65/65 [==============================] - 4s 61ms/step - loss: 0.3305
Epoch 9/15
65/65 [==============================] - 4s 61ms/step - loss: 0.3279
Epoch 10/15
65/65 [==============================] - 4s 61ms/step - loss: 0.3231
Epoch 11/15
65/65 [==============================] - 4s 61ms/step - loss: 0.3229
Epoch 12/15
65/65 [==============================] - 4s 61ms/step - loss: 0.3231
Epoch 13/15
65/65 [==================

In [10]:
model.save("my_model")

In [11]:
import shutil
shutil.make_archive('OUTPUT_NAME', 'zip', './my_model')

'/kaggle/working/OUTPUT_NAME.zip'